In [ ]:
import torch
import numpy as np
from sklearn import metrics
import torch.nn as nn
from scipy import sparse
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
import torch.nn.init as init
from scipy.linalg import expm
import math
torch.cuda.empty_cache()

In [ ]:
def load_data():
    dise_sim = np.load('data\disease_disease.npy')#138*138
    c_d = np.load('data\circRNA_disease.npy')#834*138
    c_m = np.load('data\circRNA_miRNA.npy')#834*555
    m_d = np.load('data\disease_miRNA.npy').T #555*138
    return torch.tensor(dise_sim,dtype=float),torch.tensor(c_d,dtype=torch.long),torch.tensor(c_m,dtype=torch.long),torch.tensor(m_d,dtype=torch.long)

import itertools
def calculate_sim(cd,dd):
    s1=cd.shape[0]
    ll=torch.eye(s1)
    m2=dd*cd[:,None,:]
    m1=cd[:,:,None]
    for x,y in itertools.permutations(torch.linspace(0,s1-1,s1,dtype=torch.long),2):
        x,y=x.item(),y.item()
        m=m1[x,:,:]*m2[y,:,:]
        if cd[x].sum()+cd[y].sum()==0:
            ll[x,y]=0
        else:
            ll[x,y]=(m.max(dim=0,keepdim=True)[0].sum()+m.max(dim=1,keepdim=True)[0].sum())/(cd[x].sum()+cd[y].sum())
    return ll

In [ ]:
def split_dataset(c_d,dise_sim):
    cd_axix = torch.where(c_d==1) #989   表示989对关联
    circ_x = cd_axix[0] #989
    dise_y = cd_axix[1] #989
    #有顺序的坐标
    positive_sample = torch.cat([circ_x[:,None],dise_y[:,None]],dim=1) #989*2
    rand_index1 = torch.randperm(c_d.sum()) #0~988
    #打乱顺序
    ps = torch.index_select(positive_sample,dim=0,index=rand_index1) #989*2，在上面的positive_sample中按列随机选中

    #负样本
    cd_axix_neg = torch.where(c_d==0) 
    circ_x_neg = cd_axix_neg[0]  #114103
    dise_y_neg = cd_axix_neg[1]  #114103
    #有顺序的坐标
    negative_sample = torch.cat([circ_x_neg[:,None],dise_y_neg[:,None]],dim=1) #114103*2
    rand_index2 = torch.randperm(negative_sample.shape[0])
    ns = torch.index_select(input=negative_sample,dim=0,index=rand_index2) #114103*2    

    #取与正样本相同量的负样本
    pos_len = ps.shape[0]
    train_neg = ns[0:pos_len,:] #989*2
    train_dataset = torch.cat([ps.T,train_neg.T],dim=1) #2*(989*2）

    test_dataset = torch.cat([ps.T,ns.T],dim=1)#2*(114103+989)

    #计算circ_sim
    c_sim = calculate_sim(c_d,dise_sim)
    return train_dataset,test_dataset,c_sim

dise_sim,c_d,c_m,m_d = load_data()
train_dataset,test_dataset,c_sim = split_dataset(c_d,dise_sim)
mi_sim = calculate_sim(m_d,dise_sim)

#构造无遮盖邻接矩阵
circ = torch.cat([c_sim,c_d,c_m],dim=1)
dise = torch.cat([c_d.T,dise_sim,m_d.T],dim=1)
micr = torch.cat([c_m.T,m_d,mi_sim],dim=1)
uncover_adj = torch.cat([circ,dise,micr],dim=0)
print(uncover_adj.shape)
torch.save(train_dataset,rf'data\caseStudy\train_dataset')
torch.save(test_dataset,rf'data\caseStudy\test_dataset')
torch.save(uncover_adj,rf'data\caseStudy\uncover_adj')
torch.save(c_d,rf'data\caseStudy\c_d')


In [ ]:
adj_matrix = uncover_adj
circ_sim = adj_matrix[0:834,0:834]
c_d = adj_matrix[0:834,834:834+138]

di_sim = adj_matrix[834:834+138,834:834+138]
    
c_c_c = torch.matmul(circ_sim,circ_sim)
c_d_c = torch.matmul(c_d,c_d.T)
    
d_c_d = torch.matmul(c_d.T,c_d)
d_d_d = torch.matmul(di_sim,di_sim)
torch.save(c_d,rf'data\caseStudy\cd.pth')
torch.save(circ_sim,rf'data\caseStudy\cc.pth')
torch.save(di_sim,rf'data\caseStudy\dd.pth')
torch.save(c_c_c,rf'data\caseStudy\ccc.pth')
torch.save(c_d_c,rf'data\caseStudy\cdc.pth')
torch.save(d_c_d,rf'data\caseStudy\dcd.pth')
torch.save(d_d_d,rf'data\caseStudy\ddd.pth')